In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

There are 446 pallets of varying weights (see data file) which must all be loaded onto trailers. Each trailer may carry a maximum of 20,000 pounds and no more than 14 pallets. What is the minimum number of trailers needed?

Note that to run this file, you will need to download the 'pallets.xlsx' spreadsheet in the Data folder off Github.

In [ ]:
# Read in data
p_data = pd.read_excel('Data/pallets.xlsx', index_col=0)
# Create dictionary of pallets and associated weights from the DataFrame
pallets = p_data.to_dict()['weight']

# Create 50 possible trailers which may be utilized by the model
trailers = {}
for i in range(0,50):
    trailers[i] = i

In [ ]:
# Create model
m = gp.Model('pallets2')
m.ModelSense = GRB.MINIMIZE

# Create binary decision variables representing loading a given pallet onto a given trailer
load = m.addVars(pallets, trailers, vtype=GRB.BINARY, name='load: ') 
# Create binary decision variables representing 'activation' (or ownership) of given trailer
trailer = m.addVars(trailers, vtype=GRB.BINARY, name='trailer: ')

# Our goal is to minimize the number of owned trailers
m.setObjective(gp.quicksum(trailer[j] for j in trailers))

# Add constraints that any given trailer can carry up to 20,000 pounds. 
# The (pallet * load) term represents the carried weight required for loading a given pallet onto a given trailer. 
# The (20,000 * trailer) term represents the maximum weight availability of a trailer given that the trailer is activated - a trailer must be activated to be utilized.
m.addConstrs((gp.quicksum(pallets[i] * load[i,j] for i in pallets) <= 20000 * trailer[j] for j in trailers), name='weight')

# Add constraints per pallet enforcing that a given pallet must be loaded onto a maximum of one truck exactly one time.
m.addConstrs((load.sum(i,'*') == 1 for i in pallets), name='must load ')

# Add constraints enforcing a maximum of 14 loaded pallets per given trailer.
m.addConstrs((load.sum('*',j) <= 14 for j in trailers), name='Max items for trailer ')

m.update()
# Commented to cut down on runtime
# m.display()

Set parameter Username
Set parameter LicenseID to value 2710302
Academic license - for non-commercial use only - expires 2026-09-18


In [7]:
# Optimize the chosen model
m.optimize()

# Initialize an empty array to contain the trailers utilized by the model.
used_trailers = []
# Append an auto-incremented trailer ID to the array for every utilized trailer by extracting the integer from the variable name if the variable was used. 
[used_trailers.append(int(str(j).split('[')[1].split(']')[0])) for j in m.getVars() if 'trailer' in j.varName and j.X > 0]

# Initialize an empty array to contain the utilized pallet-trailer assignments, or loads, as defined in the model.
pallet_assignments = []
# Append loads with a value of > 0, signifying utilization, to the array.
[pallet_assignments.append(str(i).split('[')[1].split(']')[0]) for i in m.getVars() if 'load' in i.varName and i.X > 0]

# Initialize an empty dictionary which will contain each trailer.
trailers_dict = {}
for j in used_trailers:
    # Initialize an empty dictionary within the overall trailers dictionary for each trailer, which will contain specific information.
    trailers_dict[j] = {}
    # Initialize a variable to count total weight.
    n = 0
    # Initialize a variable to count total number of pallets.
    x = 0
    # Initalize an array to record the pallet names assigned to the given trailer.
    a = []
    # Loop through the entire pallet_assignments array to record each trailer's assignments.
    for p in range(len(pallet_assignments)):
        # Select only the loads in the pallet_assignments array which are assigned to the given trailer being considered using an if statement. 
        # .split(',')[1] pulls out the trailer ID portion of the pallet assignment, which is then compared to the iteration, which is the trailer ID being considered. 
        if int(pallet_assignments[p].split(',')[1]) == j:
            # Add the weight of the pallet to the total weight variable. 
            # Pass the pallet name, extracted with .split(',')[0], to the pallets dictionary, as a key, to extract the weight as the assigned value.
            n += pallets[pallet_assignments[p].split(',')[0]]
            # Increase the number of pallets assigned by one. 
            x += 1
            # Append the name of the pallet (extracted as above) to the array containing all pallet names assigned to the considered trailer.
            a.append(pallet_assignments[p].split(',')[0])
    # Add the derived information to the trailer's entry in the trailers dictionary before repeating the loop for the next trailer.
    trailers_dict[j]['Total Weight'] = n
    trailers_dict[j]['# Pallets'] = x
    trailers_dict[j]['Load These Pallets'] = a
    
# Transform the dictionary into a DataFrame.
output = pd.DataFrame.from_dict(trailers_dict, orient = 'index').reset_index(drop=True)
# Add 1 to every index value so that the first trailer's ID is 1, not 0 (personal preference).
output.index = output.index + 1
# Print the DataFrame in the output.
print(output)

Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-12800H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 546 rows, 22350 columns and 66950 nonzeros
Model fingerprint: 0x6db56e9e
Variable types: 0 continuous, 22350 integer (22350 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 50.0000000
Presolve time: 0.04s
Presolved: 546 rows, 22350 columns, 66950 nonzeros
Variable types: 0 continuous, 22350 integer (22350 binary)

Root relaxation: objective 2.081995e+01, 1079 iterations, 0.02 seconds (0.03 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   20.8199